## This script finds the mean distance from a resident of an informal or atomistic area to the nearest public transit stop.

### Method:

1. Reclass ULU: [2, 3] --> [1, 1], and 0 otherwise. This gives 1 for informal/atomistic. Call this **informal**.
1. Multiply WorldPop raster by **informal**. This gives popsize only in informal pixels, zero otherwise. Call this **informal_pop**.
1. Use Overpass API to retrieve OSM transit-stop points within city bounding box.
1. Create distance raster giving distance within 20000m of OSM transit stop features. Call this **transit_distance**.
1. Multiply **informal_pop** by **transit_distance**. Call this **informal_persondistancetotransit**.
1. Sum over **informal_pop** to get total number of residents of informal/atomistic. Call this **total_informal_pop**.
1. Return **informal_persondistancetotransit** / **total_informal_pop**. This is the average distance from an informal/atomistic dwelling to nearest transit stop.


### To do:
* Return *nodata* if city not covered by ULU dataset

In [2]:
import os
import overpy
import pandas as pd
import geemap
import ee
ee.Authenticate()

Enter verification code: 4/1AbUR2VN1lKJ_dIpa2D5teKwzUj-AW-3QoMGa5I__BrSWtupZAxxYagdXGr4

Successfully saved authorization token.


In [3]:
ee.Initialize()

In [6]:
# define directory
out_dir = os.getcwd()
bucket_name = 'cities-indicators'
aws_s3_dir = "https://"+bucket_name+".s3.eu-west-3.amazonaws.com"

In [88]:
POP_YEAR = 2020

In [7]:
# get list of c4f cities
boundary_georef = pd.read_csv('https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary_georef.csv')
boundary_georef

,city_name,geo_name,aoi_boundary_name,units_boundary_name,city_boundary_name,country_code,geo_level
0,Salvador,BRA-Salvador,ADM4union,ADM4,BRA-Salvador-ADM4,BRA,ADM4
1,Bukavu,COD-Bukavu,ADM3union,ADM3,COD-Bukavu-ADM3,COD,ADM3
2,Uvira,COD-Uvira,ADM3union,ADM3,COD-Uvira-ADM3,COD,ADM3
3,Brazzaville,COG-Brazzaville,ADM4union,ADM4,COG-Brazzaville-ADM4,COG,ADM4
4,Barranquilla,COL-Barranquilla,ADM4union,ADM4,COL-Barranquilla-ADM4,COL,ADM4
5,Addis_Ababa,ETH-Addis_Ababa,ADM4union,ADM4,ETH-Addis_Ababa-ADM4,ETH,ADM4
6,Dire_Dawa,ETH-Dire_Dawa,ADM3union,ADM3,ETH-Dire_Dawa-ADM3,ETH,ADM3
7,Nairobi,KEN-Nairobi,ADM3union,ADM3,KEN-Nairobi-ADM3,KEN,ADM3
8,Antananarivo,MDG-Antananarivo,ADM4union,ADM4,MDG-Antananarivo-ADM4,MDG,ADM4
9,Mexico_City,MEX-Mexico_City,ADM2union,ADM2,MEX-Mexico_City-ADM2,MEX,ADM2


In [50]:
def get_bbox(ee_obj):
    ee_geom = ee_obj.geometry()
    coords = ee_geom.bounds().getInfo()['coordinates'][0]
    left = coords[0][0]
    bottom = coords[0][1]
    right = coords[1][0]
    top = coords[2][1]
    return (bottom, left, top, right)

In [86]:
# Get ULU polygons

ULU200 = ee.ImageCollection('projects/wri-datalab/urban_land_use/V1')
ULU4000 = ee.ImageCollection('projects/wri-datalab/cities/urban_land_use/V1')
ULUv2 = ee.ImageCollection('projects/wri-datalab/urban_land_use/V2')
ULU = ULU200.merge(ULU4000).merge(ULUv2)
ULU = ULU.select('lulc').reduce(ee.Reducer.firstNonNull()).rename('lulc') #//.clip(cityArea)//.updateMask(ULUMexico)
informal = ULU.mask(ULU.mask().gt(0)).remap([2, 3], [1, 1], 0)

In [89]:
# Get WorldPop data

pop = ee.ImageCollection('WorldPop/GP/100m/pop').filter(ee.Filter.equals('year', POP_YEAR))

In [102]:
def do_one_row(i):
    
    boundary_id_aoi = '{0}-{1}'.format(boundary_georef.loc[i, 'geo_name'], boundary_georef.loc[i, 'aoi_boundary_name'])
    boundary_path = '{0}/data/boundaries/boundary-{1}.geojson'.format(aws_s3_dir, boundary_id_aoi)
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    bbox = get_bbox(boundary_geo_ee)
    
    country_code = boundary_georef.loc[i, 'country_code']

    # Get transit points
    
    query = '('
    query += 'node[public_transport=platform]{0};'.format(str(bbox))
    query += 'node[highway=bus_stop]{0};'.format(str(bbox))
    query += 'node[highway=platform]{0};'.format(str(bbox))
    query += 'node[public_transport=stop_position]{0};'.format(str(bbox))
    query += 'node[railway=stop]{0};'.format(str(bbox))
    query += 'node[railway=platform]{0};'.format(str(bbox))
    query += 'node[station=subway]{0};'.format(str(bbox))
    query += 'node[railway=halt]{0};'.format(str(bbox))
    query += 'node[railway=tram_stop]{0};'.format(str(bbox))
    query += 'node[amenity=ferry_terminal]{0};'.format(str(bbox))
    query += 'node[aerialway=station]{0};'.format(str(bbox))
    query += 'node[amenity=ferry_terminal]{0};'.format(str(bbox))
    query += 'node[amenity=ferry_terminal]{0};'.format(str(bbox))
    query += ');out;'
    
    api = overpy.Overpass()
    result = api.query(query)
    transit_features = [ee.Feature(ee.Geometry({"type": "Point", "coordinates": [float(i.lon), float(i.lat)]})) for i in result.nodes]
    transit_featurecollection = ee.FeatureCollection(transit_features)
    
    transit_distance = transit_featurecollection.distance(20000, 10)
    
    localpop = pop.filter(ee.Filter.equals('country', country_code)).select('population').first().clip(boundary_geo_ee)
    informal_pop = localpop.multiply(informal)
    
    informal_persondistancetotransit = (informal_pop.multiply(transit_distance)).reduceRegion(reducer= ee.Reducer.sum(), geometry= boundary_geo_ee.geometry()).getInfo()['population']
    total_informal_pop = informal_pop.reduceRegion(reducer= ee.Reducer.sum(), geometry= boundary_geo_ee.geometry()).getInfo()['population']
    
    if total_informal_pop > 0:
        return informal_persondistancetotransit / total_informal_pop
    else:
        return -9999

In [103]:
do_one_row(10)

3952.5528408248188